<a href="https://colab.research.google.com/github/jonathanrbelanger-lang/Clutterfly/blob/main/JanusHamil2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
from scipy.linalg import eigh
import warnings

# --- Setup Constants ---
E_STATE = -60.0
E_TRANS = -2.00
D_LOCAL = 16

# Utility functions (re-using index_to_bits from previous cell)
def index_to_bits(k):
    b0 = k % 2
    b1 = (k // 2) % 2
    b2 = (k // 4) % 2
    b3 = (k // 8) % 2
    return np.array([b3, b2, b1, b0])

def generate_OP_matrix_new():
    """Generates O_P = sum(sigma_z_i) for 4 qubits (Same as before)."""
    P = np.zeros((D_LOCAL, D_LOCAL), dtype=np.float64)
    for k in range(D_LOCAL):
        bits = index_to_bits(k)
        diag_value = np.sum((-1)**bits)
        P[k, k] = diag_value
    return P

def generate_OF_matrix_new():
    """Generates O'_F = sigma^x_0 * sigma^z_1 * sigma^z_2 * sigma^z_3 for 4 qubits."""
    F = np.zeros((D_LOCAL, D_LOCAL), dtype=np.float64)
    for k in range(D_LOCAL):
        bits = index_to_bits(k)
        b3, b2, b1, b0 = bits

        # Calculate the Z-product factor from qubits 1, 2, 3
        z_prod_factor = (-1)**(b1 + b2 + b3)

        if b0 == 0:
            k_prime = k + 1  # Flip b0 to 1
            # F_k,k' = <0|sigma^x_0|1> * Z_prod = 1 * Z_prod
            F[k, k_prime] = z_prod_factor
            F[k_prime, k] = z_prod_factor # Symmetric since O'F is Hermitian

    return F

# --- Diagonalization ---
P_matrix = generate_OP_matrix_new()
F_matrix = generate_OF_matrix_new()
H_site_new = E_STATE * P_matrix + E_TRANS * F_matrix

# Use eigh since H is Hermitian
E_values_new, E_vectors_new = eigh(H_site_new)

print("--- Restricted D=16 Spectrum Analysis (New Hamiltonian Structure) ---")
print(f"Ground State Energy E0: {E_values_new[0]:.4f}")
print(f"Gap (0->1): {E_values_new[1] - E_values_new[0]:.4f}")
print(f"Number of unique eigenvalues found: {len(np.unique(np.round(E_values_new, 4)))}")